In [1]:
# Celda 1: Importaciones, Carga de Datos y Configuración Inicial (Actualizada)
import os
import re
import pandas as pd
import faiss
import psutil
from dotenv import load_dotenv
from tqdm.auto import tqdm
# No necesitamos joblib, ya que usaremos la persistencia de LlamaIndex
import gradio as gr
from IPython.display import display # Importado para display(df.head())
import unicodedata # <--- AÑADIDO PARA MANEJAR ACENTOS

# LlamaIndex Core y componentes
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document,
    load_index_from_storage # <--- AÑADIDO PARA CARGAR EL ÍNDICE
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.prompts import PromptTemplate
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter
from llama_index.llms.google_genai import GoogleGenAI

# --- 1. Configurar Entorno ---
num_cores = psutil.cpu_count(logical=True)
os.environ['OMP_NUM_THREADS'] = str(num_cores)
os.environ['MKL_NUM_THREADS'] = str(num_cores)
print(f"Librerías importadas. ¡Entorno listo con {num_cores} núcleos disponibles!")

# --- 2. Definir Rutas, Funciones de Normalización y Carga de Metadatos ---
METADATA_FILE = os.path.join("data", "metadatos_chatbot_final.csv")

def normalize_text(text):
    """Convierte texto a minúsculas y elimina acentos."""
    if not isinstance(text, str):
        return ""
    # Descompone los caracteres con acentos en caracter base + acento
    # y luego elimina los caracteres de acento (categoría 'Mn')
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

def load_metadata_dataframe(filepath):
    """Carga los metadatos desde un CSV a un DataFrame de Pandas."""
    try:
        df = pd.read_csv(filepath)
        df.fillna("No especificado", inplace=True)
        print("Metadatos cargados y procesados.")
        display(df.head())
        return df
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo de metadatos en la ruta: {filepath}")
        return None
    except Exception as e:
        print(f"ERROR al cargar metadatos: {e}.")
        return None

# Cargar el DataFrame al inicio
df_metadatos = load_metadata_dataframe(METADATA_FILE)

# --- CAMBIO CLAVE: Pre-normalizar columnas de autores para búsqueda eficiente ---
if df_metadatos is not None:
    print("Creando columnas normalizadas para la búsqueda de autores (insensible a acentos)...")
    df_metadatos['principal_norm'] = df_metadatos['Investigador u organismo principal'].apply(normalize_text)
    df_metadatos['equipo_norm'] = df_metadatos['Equipo de investigación'].apply(normalize_text)
    print("Columnas normalizadas creadas exitosamente.")

Librerías importadas. ¡Entorno listo con 6 núcleos disponibles!
Metadatos cargados y procesados.


,Año de publicación,Código sugerido,Nombre del archivo,Título del estudio,Categoría,Tipo de documento,Subcategoría,Idioma,Número de páginas,Incorpora perspectiva de género,...,Entidad solicitante,Entidad a cargo del estudio,Investigador u organismo principal,Equipo de investigación,Número de documento administrativo,Tipo de financiamiento,Costo del estudio,Base de datos enviada,Base pública,Url
0,1999,E99-0000,Mercado Laboral en Docencia,Análisis del Mercado de Servicios de Docencia ...,Estudios,Informe,Docentes,Español,123,No,...,Ministerio de Educación de Chile; MINEDUC,Facultad de Ciencias Económicas y Administrati...,"Bravo, David","Ruiz-Tagle, Jaime; Sanhueza, Ricardo",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
1,2000,E00-0001,Evaluación Resultados ENLACES,Diseño de un modelo de evaluación de resultado...,Estudios,Informe,TIC,Español,53,No,...,Enlaces; Ministerio de Educación de Chile; MIN...,Asesorías para el Desarrollo,"Raczynski, Dagmar","Pavez, M. Angélica",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
2,2001,E01-0001,Focalización Becas Liceos Para Todos CC,Focalización de Becas del Programa Liceo para ...,Estudios,Informe,Monitoreo y evaluación,Español,30,No,...,Ministerio de Educación de Chile; MINEDUC,"Departamento de Salud Pública, PUC","Marshall, Guillermo","Correa, Lorena",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
3,2001,E01-0002,Focalización Becas Liceos Para Todos ECO,Focalización de Becas del Programa Liceo para ...,Estudios,Informe,Monitoreo y evaluación,Español,61,No,...,Ministerio de Educación de Chile; MINEDUC,"Departamento de Salud Pública, PUC","Marshall, Guillermo","Correa, Lorena",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
4,2001,E01-0003,OFT Educación Media,Objetivos Fundamentales Transversales en la En...,Estudios,Informe,Currículo,Español,61,No,...,Ministerio de Educación de Chile; MINEDUC,"UCE, MINEDUC","Fernández, Carolina","Jashes, Jessana",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...


Creando columnas normalizadas para la búsqueda de autores (insensible a acentos)...
Columnas normalizadas creadas exitosamente.


In [2]:
# Celda 2: CONFIGURACIÓN OPTIMIZADA DE MODELOS DE IA

# Cargar la clave de API desde el archivo .env
load_dotenv()
if "OPENROUTER_API_KEY" in os.environ:
    print("Clave de API de OpenRouter cargada.")
else:
    print("¡ADVERTENCIA! No se encontró la clave de API de OpenRouter.")

if "HF_TOKEN" in os.environ:
    print("Token de Hugging Face (HF_TOKEN) encontrado en el entorno.")
else:
    print("¡ADVERTENCIA! No se encontró el token de Hugging Face.")

# --- Configuración global de LlamaIndex optimizada para CPU ---
optimal_batch_size = max(64, num_cores * 8)
print(f"Configurando el modelo de embeddings con batch size óptimo de {optimal_batch_size} para {num_cores} núcleos.")

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    embed_batch_size=optimal_batch_size,
    cache_folder="./model_cache",
    normalize=True,
    trust_remote_code=True
)

# Selecciona el proveedor de LLM aquí: "openrouter" o "gemini"
LLM_PROVIDER = "gemini"

if LLM_PROVIDER == "gemini":
    Settings.llm = GoogleGenAI(
        api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-1.5-flash-latest",
        temperature=0.1,
    )
    print("Usando Gemini como modelo LLM.")
else:
    Settings.llm = OpenRouter(
        model="google/gemma-3n-e4b-it:free",
        #model="google/gemma-3n-e2b-it:free",
        #model="mistralai/mistral-small-3.2-24b-instruct:free",
        #model="deepseek/deepseek-chat-v3-0324:free",
        #model="deepseek/deepseek-r1-0528:free",
        #model="deepseek/deepseek-chat:free",
        #model="google/gemini-2.0-flash-exp:free",
        #model="mistralai/mistral-nemo:free",
        #model="qwen/qwq-32b:free",
        #model="microsoft/mai-ds-r1:free",
        #model="meta-llama/llama-4-maverick:free",
        temperature=0.1, 
    )
    print("Usando OpenRouter como modelo LLM.")

# Función para monitorear uso de memoria
def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Uso de memoria actual: {memory_info.rss / (1024 * 1024):.2f} MB")

print("\nConfiguración de modelos completada.")
print_memory_usage()

Clave de API de OpenRouter cargada.
Token de Hugging Face (HF_TOKEN) encontrado en el entorno.
Configurando el modelo de embeddings con batch size óptimo de 64 para 6 núcleos.
Usando Gemini como modelo LLM.

Configuración de modelos completada.
Uso de memoria actual: 552.16 MB


In [5]:
# Celda 3: Creación o Carga PERSISTENTE del Índice Vectorial

# 1. Definimos la ruta donde se guardará el índice de forma permanente.
PERSIST_DIR = "./storage_index"

if os.path.exists(PERSIST_DIR):
    # Si la carpeta ya existe, cargamos el índice directamente desde ella.
    print(f"Cargando índice existente desde '{PERSIST_DIR}'...")
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    metadata_index = load_index_from_storage(storage_context)
    print("¡Índice cargado exitosamente!")

else:
    # Si la carpeta NO existe, ejecutamos el proceso de creación por primera y única vez.
    print("No se encontró un índice existente. Creando uno nuevo...")
    
    if df_metadatos is None:
        print("No se puede crear el índice porque el DataFrame de metadatos no se cargó. Revisa la Celda 1.")
        metadata_index = None
    else:
        # --- Lógica de creación (la misma que tenías antes) ---
        print("="*50)
        print("INICIANDO PROCESO DE CREACIÓN DE ÍNDICE (LENTO LA PRIMERA VEZ)...\nEste proceso solo se ejecutará una vez.")
        print("="*50)

        Settings.node_parser = SentenceSplitter(chunk_size=2048)

        # La función optimize_text_for_embedding sigue siendo útil para el contenido de los nodos
        def optimize_text_for_embedding(text, max_length=1500):
            if not isinstance(text, str): return ""
            text = text[:max_length]
            text = re.sub(r'\s+', ' ', text)  # Corregido: eliminé una barra invertida extra
            return text.strip()
            
        print("Convirtiendo metadatos en documentos de LlamaIndex...")
        metadata_documents = []
        columnas_a_incluir = [
            'Año de publicación', 'Código sugerido', 'Nombre del archivo', 'Título del estudio', 
            'Categoría', 'Tipo de documento', 'Subcategoría', 'Idioma', 'Número de páginas', 
            'Incorpora perspectiva de género', 'Año de término', 'Lugar de término', 
            'Palabras clave', 'Objetivo', 'Metodología', 'Resumen', 'Documento público', 
            'Publicación destacada', 'Publicado', 'Editorial', 'Entidad solicitante',
            'Entidad a cargo del estudio', 'Investigador u organismo principal', 'Equipo de investigación', 
            'Número de documento administrativo', 'Tipo de financiamiento', 'Costo del estudio', 
            'Base de datos enviada', 'Base pública', 'Url'
        ]
        columnas_prioritarias = ['Título del estudio', 'Resumen', 'Palabras clave', 'Objetivo', 
                                'Metodología', 'Investigador u organismo principal']

        for index, row in tqdm(df_metadatos.iterrows(), total=df_metadatos.shape[0], desc="Procesando metadatos"):
            partes_texto = []
            for col in columnas_prioritarias:
                if col in row and pd.notna(row[col]) and row[col] != "No especificado":
                    texto_optimizado = optimize_text_for_embedding(str(row[col]), 300)
                    if texto_optimizado:
                        partes_texto.append(f"{col}: {texto_optimizado}"); partes_texto.append(f"{col}: {texto_optimizado}")
            
            for col in columnas_a_incluir:
                if col not in columnas_prioritarias:
                    if col in row and pd.notna(row[col]) and row[col] != "No especificado":
                        texto_optimizado = optimize_text_for_embedding(str(row[col]), 200)
                        if texto_optimizado:
                            partes_texto.append(f"{col}: {texto_optimizado}")
            
            contenido_buscable = ". ".join(partes_texto)
            doc = Document(
                text=contenido_buscable,
                metadata={col: optimize_text_for_embedding(str(row.get(col, '')), 500) for col in columnas_a_incluir}
            )
            metadata_documents.append(doc)
        
        print("Creando índice vectorial con FAISS...")
        d = len(Settings.embed_model.get_text_embedding("test"))
        faiss_index = faiss.IndexFlatL2(d)
        vector_store = FaissVectorStore(faiss_index=faiss_index)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        
        metadata_index = VectorStoreIndex.from_documents(
            metadata_documents,
            storage_context=storage_context,
            show_progress=True 
        )
        
        # --- ¡PASO CRUCIAL! ---
        # 2. Guardamos el índice creado en la carpeta definida.
        print(f"Guardando el índice en '{PERSIST_DIR}' para uso futuro...")
        metadata_index.storage_context.persist(persist_dir=PERSIST_DIR)
        
        print("="*50)
        print("¡ÍNDICE CREADO Y GUARDADO PERMANENTEMENTE!")
        print("="*50)

No se encontró un índice existente. Creando uno nuevo...
INICIANDO PROCESO DE CREACIÓN DE ÍNDICE (LENTO LA PRIMERA VEZ)...
Este proceso solo se ejecutará una vez.
Convirtiendo metadatos en documentos de LlamaIndex...


Procesando metadatos:   0%|          | 0/837 [00:00<?, ?it/s]

Creando índice vectorial con FAISS...


Parsing nodes:   0%|          | 0/837 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/837 [00:00<?, ?it/s]

Guardando el índice en './storage_index' para uso futuro...
¡ÍNDICE CREADO Y GUARDADO PERMANENTEMENTE!


In [6]:
# Celda 4: Lógica del Chatbot e Interfaz Gradio (Con formato messages)

if 'metadata_index' not in locals() or metadata_index is None or 'df_metadatos' not in locals() or df_metadatos is None:
    print("ERROR: El índice o el DataFrame no han sido creados. Por favor, ejecuta las celdas anteriores correctamente.")
else:
    # --- GESTOR DE ESTADO Y FUNCIONES DE APOYO ---
    def get_initial_chat_state():
        return {
            "user_name": None, "current_mode": "waiting_for_name", "temp_author": None,
            "last_query_term": None, "last_results": [], "current_page": 0, "last_search_type": None
        }

    def search_by_topic(message, author_filter=None):
        print(f"Búsqueda semántica por tema: '{message}' con filtro de autor: {author_filter}")
        extraction_template = PromptTemplate("Reescribe la consulta del usuario en una lista de conceptos clave para una base de datos académica.\nConsulta: '{query_str}'\nConceptos:")
        response = Settings.llm.complete(extraction_template.format(query_str=message))
        llm_term = str(response).strip().lower()
        print(f"Término de búsqueda final: '{llm_term}'")
        retriever = metadata_index.as_retriever(similarity_top_k=50)
        retrieved_nodes = retriever.retrieve(llm_term)
        results = []
        if retrieved_nodes:
            normalized_author_filter_parts = normalize_text(author_filter).split() if author_filter else []
            for node_with_score in retrieved_nodes:
                if normalized_author_filter_parts:
                    full_text_norm = normalize_text(node_with_score.node.text)
                    if not all(part in full_text_norm for part in normalized_author_filter_parts):
                        continue
                meta = node_with_score.node.metadata
                results.append({'Título del estudio': meta.get('Título del estudio', 'N/A'),
                                'Autor(es)': f"{meta.get('Investigador u organismo principal', 'N/A')}, {meta.get('Equipo de investigación', 'N/A')}",
                                'Año de publicación': meta.get('Año de publicación', 'N/A'), 'Url': meta.get('Url', 'No disponible'),
                                '__is_semantic': True, 'score': node_with_score.score})
        results.sort(key=lambda x: x['score'], reverse=True)
        relevant_results = [res for res in results if res["score"] >= 0.40]
        return {"results": relevant_results, "query_term": llm_term}

    def search_by_author(authors_query):
        print(f"Búsqueda por autor/es: '{authors_query}'.")
        author_groups = [group.strip() for group in authors_query.split(',') if group.strip()]
        
        def df_to_results_list(df):
            df_copy = df.copy()
            df_copy['Autor(es)'] = df_copy.apply(lambda row: f"{row.get('Investigador u organismo principal', 'N/A')}, {row.get('Equipo de investigación', 'N/A')}", axis=1)
            return df_copy.to_dict('records')

        # Etapa 1: Búsqueda AND
        and_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
        for group in author_groups:
            normalized_group_parts = normalize_text(group).split()
            group_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
            for part in normalized_group_parts:
                group_mask &= (df_metadatos['principal_norm'].str.contains(part, na=False) |
                               df_metadatos['equipo_norm'].str.contains(part, na=False))
            and_mask &= group_mask
        
        and_hits_df = df_metadatos[and_mask]
        if not and_hits_df.empty:
            and_hits_df['Año de publicación num'] = pd.to_numeric(and_hits_df['Año de publicación'], errors='coerce').fillna(0)
            sorted_df = and_hits_df.sort_values(by='Año de publicación num', ascending=False)
            results_list = df_to_results_list(sorted_df)
            header = f"He encontrado {len(results_list)} documento(s) que incluyen a '{authors_query}'."
            return {"results": results_list, "header": header}

        # Etapa 2: Fallback a búsqueda OR individual
        if len(author_groups) > 1:
            response_parts = [f"No he encontrado documentos que incluyan a todos estos autores juntos. A continuación, los resultados para cada uno por separado:\n"]
            any_found = False
            for group in author_groups:
                normalized_group_parts = normalize_text(group).split()
                group_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
                for part in normalized_group_parts:
                    group_mask &= (df_metadatos['principal_norm'].str.contains(part, na=False) |
                                   df_metadatos['equipo_norm'].str.contains(part, na=False))
                
                individual_hits_df = df_metadatos[group_mask]
                
                if not individual_hits_df.empty:
                    any_found = True
                    total_individual = len(individual_hits_df)
                    limit_text = f"(mostrando los 5 más recientes de {total_individual})" if total_individual > 5 else ""
                    response_parts.append(f"\n---\n\n### Documentos de '{group}' {limit_text}:\n")
                    individual_hits_df['Año de publicación num'] = pd.to_numeric(individual_hits_df['Año de publicación'], errors='coerce').fillna(0)
                    sorted_individual_df = individual_hits_df.sort_values(by='Año de publicación num', ascending=False).head(5)
                    for i, (_, row) in enumerate(sorted_individual_df.iterrows(), 1):
                        response_parts.append(f"{i}. **Título:** {row.get('Título del estudio', 'N/A')}\n   - **Autor(es):** {row.get('Investigador u organismo principal', 'N/A')}, {row.get('Equipo de investigación', 'N/A')}\n   - **Año:** {row.get('Año de publicación', 'N/A')}\n   - **URL:** {row.get('Url', 'No disponible')}\n")
                else:
                    response_parts.append(f"\n---\n\n### Documentos de '{group}':\n\nNo se encontraron documentos.")
            
            if any_found:
                 return {"results": [], "header": "\n".join(response_parts), "is_special_format": True}

        return {"results": [], "header": f"No he encontrado documentos de '{authors_query}'. Revisa la ortografía."}

    def format_results_page(state):
        total_found = len(state["last_results"])
        if total_found == 0: return state.get("custom_header") or "No se encontraron resultados."
        MAX_TO_SHOW = 5
        start_index = (state["current_page"] - 1) * MAX_TO_SHOW
        nodes_to_show = state["last_results"][start_index : start_index + MAX_TO_SHOW]
        if not nodes_to_show: return "No hay más documentos que mostrar. Ya estás en la última página."

        header_parts = [state.get("custom_header", f"Mostrando resultados para '{state['last_query_term']}'")]
        if total_found > MAX_TO_SHOW:
            header_parts.append(f"(página {state['current_page']} de {-(-total_found // MAX_TO_SHOW)}, mostrando {len(nodes_to_show)} de {total_found} total)")
        header = " ".join(header_parts) + ":\n\n"
        
        lista_formateada = []
        for i, res in enumerate(nodes_to_show, start=start_index + 1):
            titulo = res.get('Título del estudio', 'N/A'); autores = res.get('Autor(es)', 'N/A'); ano = str(res.get('Año de publicación', 'N/A')); url = res.get('Url', 'No disponible')
            if res.get('__is_semantic'):
                score = res.get('score', 0); relevance_label = "Muy Alta" if score >= 0.75 else "Alta" if score >= 0.5 else "Media"
                relevance_str = f"**(Relevancia: {relevance_label} [{score:.0%}])**"
                lista_formateada.append(f"{i}. **Título:** {titulo} {relevance_str}\n   - **Autor(es):** {autores}\n   - **Año:** {ano}\n   - **URL:** {url}")
            else:
                 lista_formateada.append(f"{i}. **Título:** {titulo}\n   - **Autor(es):** {autores}\n   - **Año:** {ano}\n   - **URL:** {url}")

        footer = f"\n\n--- \n*Para ver más, escribe **siguiente** o `página {state['current_page'] + 1}`.*" if (start_index + MAX_TO_SHOW) < total_found else ""
        return header + "\n\n".join(lista_formateada) + footer
    
    def extract_name_from_greeting(message):
        patterns = [r"(?:soy|me llamo|mi nombre es|ll[aá]mame|dime)\s+(.+)", r"^(?:hola,? soy|hola,? mi nombre es)\s+(.+)"]
        for pattern in patterns:
            match = re.search(pattern, message, re.IGNORECASE)
            if match: return match.group(1).strip().title()
        return message.strip().title()

    with gr.Blocks(theme="default") as demo:
        chat_state = gr.State(get_initial_chat_state())
        chatbot = gr.Chatbot(
            value=[{"role": "assistant", "content": "¡Hola! Soy CemIA, asistente de IA del Centro de Estudios del MINEDUC. ¿Cómo quieres que te llame?"}], 
            type="messages",
            label="CEMIA", 
            height=500
        )
        with gr.Row(visible=False) as button_row:
            btn_author = gr.Button("Búsqueda por autoría")
            btn_topic = gr.Button("Búsqueda por temática")
            btn_combo = gr.Button("Búsqueda por autoría y temática")
            btn_bye = gr.Button("Adiós")
        user_input = gr.Textbox(placeholder="Escribe tu nombre para comenzar...", label="Tu respuesta", autofocus=True)

        def make_buttons_interactive(interactive=True):
            return gr.update(interactive=interactive), gr.update(interactive=interactive), gr.update(interactive=interactive), gr.update(interactive=interactive)

        def handle_user_input(message, history, current_state):
            # Añadir mensaje del usuario (para el formato 'messages')
            history.append({"role": "user", "content": message})
            mode = current_state["current_mode"]
            
            message_lower = message.strip().lower()
            if message_lower in ["siguiente", "más", "mas"] or re.search(r"p[aá]gina\s+(\d+)", message_lower):
                if current_state["last_results"]:
                    page_match = re.search(r"p[aá]gina\s+(\d+)", message_lower)
                    current_state["current_page"] = int(page_match.group(1)) if page_match else current_state["current_page"] + 1
                    response_content = format_results_page(current_state)
                    history.append({"role": "assistant", "content": response_content})
                else:
                    history.append({"role": "assistant", "content": "No hay resultados anteriores para paginar. Por favor, realiza una nueva búsqueda."})
                return history, current_state, gr.update(), gr.update(), gr.update(), gr.update(), gr.update(), gr.update(placeholder="Escribe 'siguiente' o elige una opción")

            button_row_update, user_input_update = gr.update(), gr.update(interactive=False, placeholder="Elige una opción de los botones")
            btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update = make_buttons_interactive(False)

            if mode == "waiting_for_name":
                user_name = extract_name_from_greeting(message)
                current_state["user_name"], current_state["current_mode"] = user_name, "waiting_for_choice"
                history.append({"role": "assistant", "content": f"¡Qué bueno tenerte aquí, {user_name}! Por favor, elige una opción."})
                button_row_update, (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = gr.update(visible=True), make_buttons_interactive(True)
            
            elif mode == "waiting_for_author":
                search_data = search_by_author(message)
                if search_data.get("is_special_format"):
                    history.append({"role": "assistant", "content": search_data["header"]})
                    current_state["last_results"] = []
                else:
                    current_state.update({"last_results": search_data["results"], "last_query_term": message, "current_page": 1, 
                                          "custom_header": search_data["header"], "last_search_type": "author"})
                    history.append({"role": "assistant", "content": format_results_page(current_state)})
                current_state["current_mode"] = "waiting_for_choice"
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                user_input_update = gr.update(placeholder="Escribe 'siguiente' o elige una opción")

            elif mode == "waiting_for_topic":
                search_data = search_by_topic(message)
                current_state.update({"last_results": search_data["results"], "last_query_term": search_data["query_term"], "current_page": 1,
                                      "custom_header": None, "last_search_type": "topic", "current_mode": "waiting_for_choice"})
                history.append({"role": "assistant", "content": format_results_page(current_state)})
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                user_input_update = gr.update(placeholder="Escribe 'siguiente' o elige una opción")
            
            elif mode == "waiting_for_combo_author":
                current_state["temp_author"], current_state["current_mode"] = message.strip(), "waiting_for_combo_topic"
                history.append({"role": "assistant", "content": "Entendido. Ahora dime la temática que quieres buscar."})
                user_input_update = gr.update(interactive=True, placeholder="Escribe la temática aquí...")
            
            elif mode == "waiting_for_combo_topic":
                author = current_state["temp_author"]
                search_data = search_by_topic(message, author_filter=author)
                current_state.update({"last_results": search_data["results"], "last_query_term": search_data["query_term"], "current_page": 1,
                                      "custom_header": f"Resultados para '{search_data['query_term']}' filtrados por el autor '{author}'", 
                                      "last_search_type": "topic", "current_mode": "waiting_for_choice", "temp_author": None})
                history.append({"role": "assistant", "content": format_results_page(current_state)})
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                user_input_update = gr.update(placeholder="Escribe 'siguiente' o elige una opción")
            
            else:
                history.append({"role": "assistant", "content": "Por favor, utiliza uno de los botones para continuar."})
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)

            return history, current_state, button_row_update, btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update, user_input_update

        def handle_choice(history, current_state, choice):
            history.append({"role": "assistant", "content": f"Has elegido: **{choice}**"})
            prompt_message, mode = "", current_state["current_mode"]
            if choice == "Búsqueda por autoría":
                mode = "waiting_for_author"
                prompt_message = "Por favor, escribe el nombre del autor/a o autores/as (separados por comas)."
            elif choice == "Búsqueda por temática":
                mode = "waiting_for_topic"
                prompt_message = "Por favor, escribe la temática que te interesa."
            elif choice == "Búsqueda por autoría y temática":
                mode = "waiting_for_combo_author"
                prompt_message = "Perfecto. Primero, escribe el nombre del autor."
            current_state["current_mode"] = mode
            history.append({"role": "assistant", "content": prompt_message})
            b1, b2, b3, b4 = make_buttons_interactive(False)
            return history, current_state, b1, b2, b3, b4, gr.update(interactive=True, placeholder="Escribe aquí...")

        def handle_bye(history, current_state):
            if current_state["user_name"]:
                history.append({"role": "assistant", "content": f"Adiós {current_state['user_name']}, ¡recuerda que siempre estoy disponible para asistirte!"})
                current_state["current_mode"] = "ended"
            b1, b2, b3, b4 = make_buttons_interactive(False)
            return history, current_state, b1, b2, b3, b4, gr.update(interactive=False, placeholder="Conversación finalizada.")

        user_input.submit(handle_user_input, [user_input, chatbot, chat_state], [chatbot, chat_state, button_row, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_author.click(lambda h, s: handle_choice(h, s, "Búsqueda por autoría"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_topic.click(lambda h, s: handle_choice(h, s, "Búsqueda por temática"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_combo.click(lambda h, s: handle_choice(h, s, "Búsqueda por autoría y temática"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_bye.click(handle_bye, [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])

    print("\nLanzando la interfaz de CEMIA con botones...")
    demo.launch(inline=True, share=False)


Lanzando la interfaz de CEMIA con botones...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Búsqueda semántica por tema: 'hay documentos sobre enseñanza básica?' con filtro de autor: None
Término de búsqueda final: 'conceptos:

* **educación básica:**  (o educación primaria, educación elemental, dependiendo del contexto geográfico)
* **documentación:** (o  publicaciones, artículos, libros, informes, etc.)
* **enseñanza:** (o pedagogía, didáctica, instrucción)'


Traceback (most recent call last):
  File "c:\Users\fburg\Videos\Proyecto-Diplomatura-ChatBot\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\fburg\Videos\Proyecto-Diplomatura-ChatBot\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\fburg\Videos\Proyecto-Diplomatura-ChatBot\.venv\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\fburg\Videos\Proyecto-Diplomatura-ChatBot\.venv\Lib\site-packages\gradio\blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignor

Búsqueda por autor/es: 'fabian burgos'.


C:\Users\fburg\AppData\Local\Temp\ipykernel_22148\2759400171.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  and_hits_df['Año de publicación num'] = pd.to_numeric(and_hits_df['Año de publicación'], errors='coerce').fillna(0)
